# pandas层次化索引

### 1.创建多层行索引

##### 1)隐式构造
最常见的方法是给DataFrame构造函数的index参数传递两个或更多的数组

Series也可以创建多层索引

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
# 隐式创建
s = Series(data=[1,2,3,"a"],index=[['a','a','b','b'],["期中","期末","期中","期末"]])
s

a  期中    1
   期末    2
b  期中    3
   期末    a
dtype: object

In [3]:
df = DataFrame(data = [1,2,3,4],
              index = [["a","a","b","b"],['期中',"期末","期中","期末"]],
              columns = ["Python"])
df

Python
a 期中       1
  期末       2
b 期中       3
  期末       4

In [4]:
# 三层行索引
df = DataFrame(data = np.random.randint(0,150,size = 8),
              index = [["a","a","a","a","b","b","b","b"],
                       ["期中","期中","期末","期末","期中","期中","期末","期末"],
                       ["一单元","二单元","一单元","二单元","一单元","二单元","一单元","二单元"]],
              columns = ["Python"])
df

Python
a 期中 一单元     116
     二单元      66
  期末 一单元     100
     二单元      74
b 期中 一单元      91
     二单元     104
  期末 一单元      97
     二单元     110

##### 2)显示构造pd.MultiIndex
    使用数组

In [5]:
df1 = DataFrame(data = np.random.randint(0,150,size = 8),
               index = pd.MultiIndex.from_arrays([["a","a","a","a","b","b","b","b"],
                       ["期中","期中","期末","期末","期中","期中","期末","期末"],
                       ["一单元","二单元","一单元","二单元","一单元","二单元","一单元","二单元"]]),
               columns = ["Python"])
df1

Python
a 期中 一单元      83
     二单元      87
  期末 一单元     127
     二单元      39
b 期中 一单元      36
     二单元      87
  期末 一单元      44
     二单元      35

    使用tuple

In [6]:
df3 = DataFrame(np.random.randint(0,150,size = 4), 
               index = pd.MultiIndex.from_tuples([("a","期中"),("a","期末"),("b","期中"),("b","期末")]),
               columns = ["Python"])
df3

Python
a 期中      91
  期末      93
b 期中      84
  期末      18

    使用product
    最简单，推荐使用

In [7]:
df4 = DataFrame(np.random.randint(0,150,size = (8,2)),
               index = pd.MultiIndex.from_product([list("abcd"),["期中","期末"]]),
               columns = ['Python',"高数"])
df4

Python   高数
a 期中      15   39
  期末      56   14
b 期中      56   28
  期末     118    6
c 期中      10   94
  期末     106   77
d 期中      97  147
  期末      67   68

### 2.多层列索引
除了行索引index，列索引columns也能用同样的方法创建多层索引

In [8]:
#三层列索引
df = DataFrame(data = np.random.randint(0,150,size = (1,8)),
              columns = [["a","a","a","a","b","b","b","b"],
                       ["期中","期中","期末","期末","期中","期中","期末","期末"],
                       ["一单元","二单元","一单元","二单元","一单元","二单元","一单元","二单元"]],
              index = ["Python"])
df

a                 b              
        期中      期末        期中       期末     
       一单元 二单元 一单元  二单元  一单元 二单元  一单元  二单元
Python  32  89  52  117  100   3  135  140

### 3.多层索引对象的索引与切片操作
#### 1）Series的操作
[重要]对于Series来说，直接中括号 [ ] 与使用.loc()完全一样，因此，推荐使用中括号索引和切片。

(1) 索引

In [9]:
s

a  期中    1
   期末    2
b  期中    3
   期末    a
dtype: object

In [10]:
s["a"]["期末"]

2

In [12]:
s["a","期末"]

2

(2)切片

In [13]:
s["a":"b"]

a  期中    1
   期末    2
b  期中    3
   期末    a
dtype: object

In [14]:
s["期中":"期末"]

Series([], dtype: object)

In [15]:
# iloc 对于多层索引更好一点， 更灵活点
s.iloc[0:3]

a  期中    1
   期末    2
b  期中    3
dtype: object

#### 2）DataFrame的操作
(1) 可以直接使用列名称来进行列索引

(2) 使用行索引需要用loc()等函数

【极其重要】推荐使用loc()函数

注意在对行索引的时候，若一级行索引还有多个，对二级行索引会遇到问题！也就是说，无法直接对二级索引进行索引，必须让二级索引变成一级索引后才能对其进行索引！

In [16]:
df4

Python   高数
a 期中      15   39
  期末      56   14
b 期中      56   28
  期末     118    6
c 期中      10   94
  期末     106   77
d 期中      97  147
  期末      67   68

In [17]:
df4.loc["a"]["Python"]["期中"]

15

In [23]:
df4.loc["a","期中"].loc["Python"]  # 行与列必须分清

15

In [24]:
# 切片
df4["a":"c"]

Python  高数
a 期中      15  39
  期末      56  14
b 期中      56  28
  期末     118   6
c 期中      10  94
  期末     106  77

In [26]:
df4.iloc[0:3]

Python  高数
a 期中      15  39
  期末      56  14
b 期中      56  28

### 4. 索引的堆（stack）

In [27]:
df4

Python   高数
a 期中      15   39
  期末      56   14
b 期中      56   28
  期末     118    6
c 期中      10   94
  期末     106   77
d 期中      97  147
  期末      67   68

In [29]:
# 将最里面的行索引变成了列索引
df4.unstack()

Python        高数    
      期中   期末   期中  期末
a     15   56   39  14
b     56  118   28   6
c     10  106   94  77
d     97   67  147  68

In [30]:
df4.unstack(level=0)

Python                高数             
        a    b    c   d   a   b   c    d
期中     15   56   10  97  39  28  94  147
期末     56  118  106  67  14   6  77   68

In [31]:
# 把列索引变成了行索引
df.stack(level=1)

a         b     
          一单元  二单元  一单元  二单元
Python 期中  32   89  100    3
       期末  52  117  135  140

In [32]:
df.stack(level=(0,2))

期中   期末
Python a 一单元   32   52
         二单元   89  117
       b 一单元  100  135
         二单元    3  140

In [ ]:
"""
小技巧:
        使用stack()的时候，level等于哪一个，哪一个就消失，出现在行里。
        使用unstack()的时候，level等于哪一个，哪一个就消失，出现在列里。
"""

### 5. 聚合操作
【注意】

需要指定axis

【小技巧】和unstack()相反，聚合的时候，axis等于哪一个，哪一个就保留。

所谓的聚合操作：平均数，方差，最大值，最小值……

In [34]:
df4

Python   高数
a 期中      15   39
  期末      56   14
b 期中      56   28
  期末     118    6
c 期中      10   94
  期末     106   77
d 期中      97  147
  期末      67   68

In [35]:
df4.div(10)

Python    高数
a 期中     1.5   3.9
  期末     5.6   1.4
b 期中     5.6   2.8
  期末    11.8   0.6
c 期中     1.0   9.4
  期末    10.6   7.7
d 期中     9.7  14.7
  期末     6.7   6.8

In [36]:
# where函数（条件）
df4.where(df4 > 5,other=100)

Python   高数
a 期中      15   39
  期末      56   14
b 期中      56   28
  期末     118    6
c 期中      10   94
  期末     106   77
d 期中      97  147
  期末      67   68

In [37]:
df4.sum()

Python    525
高数        473
dtype: int64

In [ ]:
df4